In [1]:
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
T = 20
N = 200
D = 32   # obs dim
H = 128
x = tf.placeholder(dtype=tf.float32, shape=(T, N, D))
x

<tf.Tensor 'Placeholder:0' shape=(20, 200, 32) dtype=float32>

In [4]:
gru = tf.contrib.cudnn_rnn.CudnnGRU(num_layers=1, num_units=128)

In [34]:
init = np.zeros([1, 10, N, H])
init.shape

(1, 10, 200, 128)

In [35]:
s0 = tuple(init)

In [36]:
s0[0].shape

(10, 200, 128)

## With bulid

In [16]:
gru.build(input_shape=(T, N, D))
gru.call(x, initial_state=init)

(<tf.Tensor 'CudnnRNN_6:0' shape=(20, 200, 32) dtype=float32>,
 (<tf.Tensor 'CudnnRNN_6:1' shape=(200, 128) dtype=float32>,))

In [17]:
gru.call(x)

(<tf.Tensor 'CudnnRNN_7:0' shape=(20, 200, 128) dtype=float32>,
 (<tf.Tensor 'CudnnRNN_7:1' shape=(1, 200, 128) dtype=float32>,))

## Without build

In [9]:
gru_ = tf.contrib.cudnn_rnn.CudnnGRU(num_layers=1, num_units=128)

In [10]:
T = 20
N = 200
D = 32   # obs dim
H = 128
x_ = tf.placeholder(dtype=tf.float32, shape=(T, N, D))

In [18]:
gru_(x_)

(<tf.Tensor 'cudnn_gru_2/CudnnRNN_1:0' shape=(20, 200, 128) dtype=float32>,
 (<tf.Tensor 'cudnn_gru_2/CudnnRNN_1:1' shape=(1, 200, 128) dtype=float32>,))

In [50]:
s0 = tf.placeholder(shape=(1, N, H), dtype=tf.float32)

In [51]:
gru_.call(x_, initial_state=(s0,))

(<tf.Tensor 'CudnnRNN_10:0' shape=(20, 200, 128) dtype=float32>,
 (<tf.Tensor 'CudnnRNN_10:1' shape=(1, 200, 128) dtype=float32>,))

In [56]:
s1 = tf.zeros([3,4])
tf.expand_dims(s0, 0)

<tf.Tensor 'ExpandDims_4:0' shape=(1, 1, 200, 128) dtype=float32>

In [58]:
tf.transpose(x_, (1, 0, 2))

<tf.Tensor 'transpose_3:0' shape=(200, 20, 32) dtype=float32>

In [59]:
x_

<tf.Tensor 'Placeholder_1:0' shape=(20, 200, 32) dtype=float32>

In [60]:
x_[0]

<tf.Tensor 'strided_slice_11:0' shape=(200, 32) dtype=float32>

## data pipeline

In [47]:
class ReplayBuffer:
    """
    A simple FIFO experience replay buffer for SAC agents.
    """

    def __init__(self, obs_dim, act_dim, size, h_size, seq_length, flag="single", normalize=False):
        self.flag = flag
        self.normalize = normalize
        self.sequence_length = seq_length
        self.ptr, self.size, self.max_size = 0, 0, size
        self.obs_dim = obs_dim
        size += seq_length  # in case index is out of range
        self.obs1_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.hidden_buf = np.zeros([size, h_size], dtype=np.float32)
        self.acts_buf = np.zeros([size, act_dim], dtype=np.float32)
        self.rews_buf = np.zeros([size, 1], dtype=np.float32)
        self.done_buf = np.zeros([size, 1], dtype=np.float32)
        self.target_done_ratio = 0

    def store(self, obs, s_t_0, act, rew, done):
        self.obs1_buf[self.ptr] = obs / 10 if self.normalize else obs
        self.hidden_buf[self.ptr] = s_t_0
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size  # 1%20=1 2%20=2 21%20=1
        self.size = min(self.size + 1, self.max_size)  # use self.size to control sample range
        self.target_done_ratio = np.sum(self.done_buf) / self.size

    def sample_batch(self, batch_size=32):
        """
        :param batch_size:
        :return: s a r s' d
        """
        while 1:
            idxs_c = np.empty([batch_size, self.sequence_length])  # N T+1

            # for i in range(batch_size):
            end = False
            while not end:
                ind = np.random.randint(0, self.size - 5)  # random sample a starting point in current buffer
                idxs = np.arange(ind, ind + self.sequence_length)  # extend seq from starting point
                # 0000000 or 0000010 is valid   sum(done) <= 1
                is_valid_pos = True if sum(self.done_buf[idxs]) == 0 else \
                    (((self.sequence_length - np.where(self.done_buf[idxs] == 1)[0][0]) == 2) and
                     (sum(self.done_buf[idxs]) == 1))

                end = True if is_valid_pos else False

            idxs_c[i] = idxs

            np.random.shuffle(idxs_c)
            idxs = idxs_c.astype(int)
            # print(self.target_done_ratio, np.sum(self.done_buf[idxs]) / batch_size)
            data = dict(obs1=self.obs1_buf[idxs],
                        s_t_0=self.hidden_buf[idxs][:, 0, :],  # slide N T H to N H
                        acts=self.acts_buf[idxs],
                        rews=self.rews_buf[idxs],
                        done=self.done_buf[idxs])
            yield data

In [48]:
replay_buffer = ReplayBuffer(obs_dim=64,
                                 act_dim=4,
                                 size=int(1e4),
                                 h_size=128,
                                 seq_length=20,
                                 flag="seq",
                                 normalize=False)

In [49]:
for _ in range(int(1e4)):
    replay_buffer.store(np.random.random([1, 64]), np.random.random([1, 128]), np.random.random([1, 4]), 0, 0)

In [50]:
ds = tf.data.Dataset.from_generator(replay_buffer.sample_batch, output_types=tf.float32)

In [51]:
ds.batch(32)

<BatchDataset shapes: <unknown>, types: tf.float32>

In [54]:
dataset = tf.data.Dataset.range(10)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
sess = tf.Session()
for i in range(100):
  value = sess.run(next_element)
  print(value)

0
1
2
3
4
5
6
7
8
9


OutOfRangeError: End of sequence
	 [[node IteratorGetNext_6 (defined at <ipython-input-54-f75211e63125>:3)  = IteratorGetNext[output_shapes=[[]], output_types=[DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_6)]]

Caused by op 'IteratorGetNext_6', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/gu/project/env_3_6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/gu/project/env_3_6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/gu/project/env_3_6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/gu/project/env_3_6/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, in start
    